In [ ]:
import platform

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

import bs4, time, requests
import pandas as pd

pd.options.display.max_rows = None
pd.options.display.max_columns = None
import numpy as np

import plotly
from plotly.offline import plot_mpl
import plotly.offline as py
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=False)

In [19]:
# Pega todos os resultados do google
class socialMediaScrapper():
    def __init__(self, has_data=False):
        if has_data != True: self.driver_() #inicia o navegador

    def __call__(self):
        #ao chamar como funcao ira fazer fazer a pesquisa, pegar os dados das redes sociais e tambem de anuncios
        print('getting search results...')
        self.saude_curitiba = self.googleSearch()
        print('findind social media accounts...')
        self.saude_curitiba = self.findSocialMediaAccounts(self.saude_curitiba)
        print('getting instagram data...')
        self.saude_curitiba = self.getInstagramData(self.saude_curitiba)
        print('getting facebook data...')
        self.saude_curitiba = self.getFacebookData(self.saude_curitiba)
        print('getting ads data...')
        self.saude_curitiba = self.getAdsData(self.saude_curitiba)
        print('all done')
        return self.saude_curitiba
    
    def driver_(self):
        options = webdriver.ChromeOptions()
        options.add_argument("profile")
        options.add_argument("user-data-dir=./Browser/Profile")
        
        if "Windows" in platform.platform():
            executable_path = './Browser/Driver/Win/chromedriver.exe' 
            print('Platform: Windows')
        else:
            executable_path = './Browser/Driver/chromedriver'
            
        self.driver = webdriver.Chrome(executable_path=executable_path, options=options)

        
        
    def nextPage(self):
        # Procura o botao para a próxima página
        htmlPage = bs4.BeautifulSoup(self.driver.page_source, features='html.parser')

        button_status = htmlPage.find('button', {'jsaction':'pane.paginationSection.nextPage'})
        button_next = htmlPage.find('span', {'class':'n7lv7yjyC35__button-next-icon'})
        if button_status != None and button_status.get('disabled') == 'true':
            return 'end'
        elif button_next != None:
            return 'next'

    def typeResultList(self):
        ''' encontra qual o tipo de resultados que o google devolveu
            * ha resultados com o link direto
            * ha resultados que precisa de um clique para acessar os dados
        '''
        htmlPage = bs4.BeautifulSoup(self.driver.page_source, features='html.parser')
        route_button = htmlPage.find('div',{'class':'section-result-action-icon-container'})
        if route_button != None:
            return 'simple'
        else:
            return 'need click'
    
    def googleSearch(self, url=None, clean=True):
        '''
            * pode ser chamado sem url desde que esteja na aba da pesquisa
            * clean=True para retornar os dados ja limpes pelo modulo ProcessData
        '''
        if url == None:
            input('Aperte Enter quando já estiver na pesquisa que deseja capturar os dados: ')
        else:
            self.driver.get(url)
            time.sleep(2)

        # Verifica qual o tipo de resultados
        if self.typeResultList() == 'need click':
            # importa a outra classe para usar
            moreActions = self.moreBrowseActions(self.driver)
            simpleScrapp = False
        else:
            simpleScrapp  = True
        
        results = []
        while True:
            try:
                htmlPage = bs4.BeautifulSoup(self.driver.page_source, features='html.parser')
                if simpleScrapp == True:
                    results.extend(htmlPage.find_all('div', {'class':'section-result'}))
                else:
                    click_error = 0
                    for i in range(1, 21):
                        if moreActions.clickResult(i) == False: click_error+=1
                        if click_error >=2: break
                        
                        time.sleep(2)
                        results.append(moreActions.dataFromClickedResult(i))
                        moreActions.backToResults()
                        time.sleep(1)
                        
                        
                if self.nextPage() == 'end': 
                    break
                else:
                    next_page = self.driver.find_element_by_class_name('n7lv7yjyC35__button-next-icon')
                    next_page.click()
                    time.sleep(2)
            except KeyboardInterrupt: 
                break
            except Exception as error:
                if htmlPage.find('div',{'class':'section-no-result noprint'}) != None: 
                    break
                else:
                    print('ERROR IN [socialMediaScrapper()] function: googleSearch()\n')
                    print(type(error), str(error))
        
        if simpleScrapp == False:
            results = pd.DataFrame(results)
            results.reset_index(drop=True, inplace=True)
            return results
        elif clean == False:
            return list(set(results))
        else:
            clean = ProcessData()
            return clean.googleSearchResults(results)
        
    class moreBrowseActions():
        ''' acoes extras para o navegador caso os resultados nao estejam na lista e precise ser clicado'''
        def __init__(self, driver):
            self.driver = driver
            
        def clickResult(self, position):
            ''' ira clicar no resultado e extrair os dados'''
            try:
                self.driver.find_element_by_xpath('//div[@data-result-index=' + str(position) + ']').click()
                return True
            except Exception as error:
                if 'NoSuchElementException' in str(type(error)): return False
                
                print('ERROR IN [socialMediaScrapper()] CLASS-> {moreBrowseActions} function: clickResult()\n')
                print(type(error), str(error))
                return False

        def backToResults(self):
            try:
                self.driver.find_element_by_xpath('//button[@class="section-back-to-list-button blue-link noprint"]').click()
                return True
            except Exception as error:
                if 'NoSuchElementException' in str(type(error)): return False
                
                print('ERROR IN [socialMediaScrapper()] CLASS-> {moreBrowseActions} function: backToResults()\n')
                print(type(error), str(error))
                return False

        def dataFromClickedResult(self, position):
            htmlPage = bs4.BeautifulSoup(self.driver.page_source, features='html.parser')

            fone, link, address = None, None, None
            for r in htmlPage.find_all('div',{'class':'ugiz4pqJLAG__root ugiz4pqJLAG__one-line-text ugiz4pqJLAG__dense ugiz4pqJLAG__metadata-shown-on-hover-only ugiz4pqJLAG__clickable'}):
                if r.find('img').get('src') == '//www.gstatic.com/images/icons/material/system_gm/1x/place_gm_blue_24dp.png': address = r.text 
                if r.find('img').get('src') == '//www.gstatic.com/images/icons/material/system_gm/1x/public_gm_blue_24dp.png': link = r.text
                if r.find('img').get('src') == '//www.gstatic.com/images/icons/material/system_gm/1x/phone_gm_blue_24dp.png': fone = r.text

            name = htmlPage.find('h1',{'class':'section-hero-header-title-title GLOBAL__gm2-headline-5'})
            if name != None and name.find('span') != None: name = name.find('span').text
            details = htmlPage.find('button',{'jsaction':'pane.rating.category'})
            if details != None: details = details.text
            classification = htmlPage.find('span',{'class':'section-star-display'})
            if classification != None: classification = classification.text

            return {'classification': classification, 'position': position, 'name': name, 'fone': fone, 'address': address, 'link': link, 'details': details}
    
    def findSocialMediaAccounts(self, data_business, print_process=True):
        ''' recebe uma lista ou dataframe e procura pelas redes sociais em seus sites'''
        business_df = data_business

        assert str(type(business_df)) == "<class 'pandas.core.frame.DataFrame'>", 'o input deve ser um pandas.dataframe'

        for row, link in business_df[business_df['link'].isnull() == False]['link'].T.iteritems():
            try:
                req = requests.get(link)
                if req.status_code == 200:
                    htmlPage = bs4.BeautifulSoup(req.content, features='html.parser', from_encoding="iso-8859-1")

                    business_df.loc[row, 'has_instagram'] = True if str(req.content).find('instagram') != -1 else False
                    business_df.loc[row, 'has_facebook'] = True if str(req.content).find('facebook') != -1 else False

                    for link in htmlPage.find_all('a'):
                        if link.get('href') != None and 'facebook' in link.get('href'):
                            business_df.loc[row, 'facebook_link'] = link.get('href')
                        elif link.get('href') != None and 'instagram' in link.get('href'):
                            business_df.loc[row, 'instagram_link'] = link.get('href')

                    if print_process == True and row % 100 == 0:
                        print('{} processed'.format(row))
                else:
                    business_df.loc[row, 'link_problem'] = True 
            except Exception as error:
                business_df.loc[row, 'link_problem'] = True
        return business_df
    
    def getInstagramData(self, data_business, print_process=True, clean=True):
        ''' Ira pegar os dados gerais das contas de Instagram encontradas '''
        data = data_business

        assert str(type(data)) == "<class 'pandas.core.frame.DataFrame'>", 'o input deve ser um pandas.dataframe'    
        # verifica se ha navegador aberto, se nao houver ira abrir
        try:
            self.driver.current_url
        except Exception as error: 
            if 'chrome not reachable' or 'HTTPConnectionPool' in str(error):
                self.driver_()

        for row, link in data[data['instagram_link'].isnull() == False]['instagram_link'].T.iteritems():
            first_w = data.loc[row]['instagram_link'].find('w')
            content = data.loc[row]['instagram_link']
            data.loc[row]['instagram_link'] = 'https://' + content[first_w:]
            
            try:
                self.driver.get(link)
                htmlPage = bs4.BeautifulSoup(self.driver.page_source, features='html.parser', from_encoding="iso-8859-1")

                # PEGA OS DADOS NUMERICOS DA PAGINA
                numbers = htmlPage.find_all('a', {'class':'-nal3'})
                if numbers != None:
                    data.loc[row, 'instagram_posts'] = numbers[0].text
                    data.loc[row, 'instagram_followers'] = numbers[1].text
                    data.loc[row, 'instagram_following'] = numbers[2].text

                #PEGA OS DADOS DE TEXTO DA PAGINA
                data_text = htmlPage.find('div', {'class':'-vDIg'})
                page_name = data_text.find('h1', {'class':'rhpdm'})
                if page_name != None: data.loc[row, 'instagram_page_name'] = page_name.text
                bio_text = data_text.find('span')   
                if bio_text != None: data.loc[row, 'instagram_bio_text'] = bio_text.text
                link = data_text.find('a',{'class':'yLUwa'})
                if link != None: data.loc[row, 'instagram_bio_link'] = link.get('href')
                username = htmlPage.find('h2', {'class':'_7UhW9'})
                if username != None: data.loc[row, 'instagram_username'] = username.text

                if print_process == True: print('getting Instagram Data, {} processed, last username saved: {}'.format(row, username.text))
                time.sleep(1)
            except Exception as error:
                if 'IndexError' in str(type(error)):
                    unavilable = htmlPage.find('div',{'class':'_07DZ3'})
                    data.loc[row, 'instagram_link_problem'] = True if unavilable != None and "Sorry, this page isn't available." in unavilable.text else None
                    continue 
                else:
                    data.loc[row, 'instagram_link_problem'] = '404'
        
        if clean == False:
            return data
        else:
            clean = ProcessData()
            return clean.cleanInstagramData(data)
        
    def getFacebookData(self, data_business, print_process=True, clean=True):
        ''' Ira pegar os dados gerais das contas de Facebook encontradas '''
        data = data_business

        assert str(type(data)) == "<class 'pandas.core.frame.DataFrame'>", 'o input deve ser um pandas.dataframe'    
        # verifica se ha navegador aberto, se nao houver ira abrir
        try:
            self.driver.current_url
        except Exception as error: 
            if 'chrome not reachable' or 'HTTPConnectionPool' in str(error):
                self.driver_()
        
        for row, link in data[data['facebook_link'].isnull() == False]['facebook_link'].T.iteritems():
            try:
                self.driver.get(link)
                htmlPage = bs4.BeautifulSoup(self.driver.page_source, features='html.parser')

                facebook_page_name = htmlPage.find('span', {'class':'_33vv'})
                if facebook_page_name != None: data.loc[row, 'facebook_page_name'] = facebook_page_name.text

                facebook_page_user = htmlPage.find('a', {'class':'_2wmb'})
                if facebook_page_user != None: data.loc[row, 'facebook_page_user'] = facebook_page_user.text

                page_data = htmlPage.find('div', {'class':'_4-u2 _6590 _3xaf _4-u8'})
                
                if page_data != None:
                    saved = []
                    for div in page_data.find_all('div'):
                        if div.text not in saved:
                            if 'follow' in div.text or 'seguindo ' in div.text:
                                data.loc[row, 'facebook_followers'] = div.text
                            elif 'like' in div.text or 'curtiram' in div.text:
                                data.loc[row, 'facebook_likes'] = div.text
                            elif 'check' in div.text:
                                data.loc[row, 'check-ins'] = div.text
                else:
                    data.loc[row, 'facebook_link_problem'] = True

                about_content = htmlPage.find('div', {'class':'_4-u2 _u9q _3xaf _4-u8'})
                data.loc[row, 'about_content'] = str(about_content)

                time.sleep(1)
                if print_process == True: print('getting Facebook Data, {} processed'.format(row))
            except Exception as error:
                print(type(error), print(error))
                data.loc[row, 'facebook_link_problem'] = True

        if clean == False:
            return data
        else:
            clean = ProcessData()
            return clean.cleanFacebookData(data)
        
    def getAdsData(self, data_business, print_process=True, clean=True):
        ''' Ira pegar os dados de anuncios encontrados '''
        self.dataAds = data_business
        
        assert str(type(data_business)) == "<class 'pandas.core.frame.DataFrame'>", 'o input deve ser um pandas.dataframe'    
        # verifica se ha navegador aberto, se nao houver ira abrir
        try:
            sp.driver.current_url
        except Exception as error: 
            if 'chrome not reachable' or 'HTTPConnectionPool' in str(error):
                sp.driver_()

        sp.driver.get('https://pt-br.facebook.com/ads/library')
        time.sleep(1)

        for row, content in self.dataAds.loc[(self.dataAds['facebook_page_name'].isnull() == False)][['facebook_page_name', 'facebook_page_user']].T.iteritems():
            clicked = self.searchPageName(content['facebook_page_name'], content['facebook_page_user'])
            
            if clicked == False: 
                self.dataAds.loc[row, 'ads_page_problem'] = '404' #rodou todos os resultados e nao encontrou a pagina
            elif clicked == True or clicked == 'By_page_name':
                if not(self.getDataAdsPage(row)): 
                    self.dataAds.loc[row, 'ads_page_problem'] = True #por alguma razao nao encontrou os dados
                elif clicked == 'By_page_name':
                    self.dataAds.loc[row, 'ads_page_problem'] = 'By_page_name'

                time.sleep(1)
                print('{} pages processed'.format(row))
        
        if clean == False:
            return self.dataAds
        else:
            clean = ProcessData()
            return clean.cleanAdsData(self.dataAds)

    def clickInPageResultList(self, facebook_page_info, by_page_name=False):
        '''
            procura a pagina na lista de resultados e se algum resultado na lista tiver o mesmo @usuario ira clicar
        '''

        div_to_find = '_8t5z' if by_page_name == False else '_7h1e'
        results = sp.driver.find_elements_by_class_name(div_to_find)

        if results != None:
            for result in results:
                content_to_verify = result.text[:result.text.find(' ')] if by_page_name == False else result.text
                # se nao for pelo nome da pagina, ira pegar o nome de usuario que esta no inicio do texto

                if facebook_page_info == content_to_verify:
                    result.click()
                    time.sleep(3)
                    return True
            return False
        else:
            return 'Not Found'

    def searchPageName(self, facebook_page_name, facebook_page_user):
        '''
            Recebe uma pagina e seu nome_de_usuario e pesquisa na biblioteca de anuncio
        '''
        try:
            search_box = sp.driver.find_element_by_class_name('_7hgq')
            search_box.click()

            for char in search_box.get_attribute('value'):
                search_box.send_keys(Keys.BACKSPACE)
                #apaga qualquer conteudo escrito na barra de pesquisa

            search_box.send_keys(facebook_page_name)
            time.sleep(2)

            clicked = self.clickInPageResultList(facebook_page_user)

            # SE NAO ENCONTROU A PAGINA NA LISTA IRA VERIFICAR SE HA OPCAO VER MAIS E SE ESTA MAIS ABAIXO NOS RESULTADOS
            if clicked == False:
                try:
                    more_results = more_results = sp.driver.find_element_by_class_name('_7h65')
                    more_results.click()
                    time.sleep(1)
                except:
                    pass

                if self.clickInPageResultList(facebook_page_user):
                    return True
                else:
                    # se ainda assim nao encontrar a pagina ira tentar pelo nome da pagina
                    if self.clickInPageResultList(facebook_page_name, by_page_name=True):
                        return 'By_page_name'
                    else:
                        False
            else:
                return True
        except Exception as error:
            print('error in: [searchPageName]: \n')
            print(type(error), error)
            return 'Error'

    def getDataAdsPage(self, dataframe_row):
        htmlPage = bs4.BeautifulSoup(sp.driver.page_source, features='html.parser')
        self.dataAds.loc[dataframe_row, 'library_ads_url'] = sp.driver.current_url
        time.sleep(1)

        active_ads = htmlPage.find('div',{'class':'_7gn2'})
        if active_ads != None: self.dataAds.loc[dataframe_row, 'active_ads'] = active_ads.text

        page_data = htmlPage.find('div',{'class':'_3-8- _3qn7 _61-0 _2fyh _3qnf'})
        if page_data != None:
            for row_in_list in page_data.find_all('div'):
                # percorre as linhas ate encontrar o icone de pagina criada ira pegar o conteudo
                if row_in_list.find('i', {'class': 'img sp_9Pca48nsujv sx_50d5dd'}) != None: 
                    created_in = row_in_list.find('span',{'class':'_3-99'})
                    if created_in != None: self.dataAds.loc[dataframe_row, 'created_in'] = created_in.text
                    return True
                else:
                    self.dataAds.loc[dataframe_row, 'created_in'] = '404' #por alguma razao nao encontrou os dados entao ira retornar False
        else:
            self.dataAds.loc[dataframe_row, 'created_in'] = '404' #por alguma razao nao encontrou os dados entao ira retornar False

In [20]:
class ProcessData():
    def __init__(self):
        pass
    
    def googleSearchResults(self, results_list):
        ''' * reomocao de duplicatos pelo endereco '''
        
        self.data = [] #sera o dataframe com os dados
        
        #recebe a lista de resultados do google e processa os dados
        for count, r in enumerate(results_list):
            name = r.find('h3', {'class':'section-result-title'})
            if name != None: name = name.text
            address = r.find('span', {'class':'section-result-location'})
            if address != None: address = address.text
            details = r.find('span', {'class':'section-result-details'})
            if details != None: details = details.text
            fone = r.find('span', {'class':'section-result-info section-result-phone-number'})
            if fone != None: fone = fone.text
            link = r.find('a')
            if link != None: link = link.get('href')
            classification = r.find('span', {'class':'cards-rating-score'})
            if classification != None: classification = classification.text

            data_result = {'classification': classification, 'position': count+1, 'name': name, 'fone': fone, 'address': address, 'link': link, 'details': details}
            self.data.append(data_result)
    
        self.data = pd.DataFrame(self.data)
        self.data.drop_duplicates(['address']) 
        self.data.reset_index(drop=True, inplace=True)
        return self.data
    
    def cleanInstagramData(self, business_df):
        ''' Limpa os dados coletados do Instagram '''
        data_instagram_page = business_df
        assert str(type(data_instagram_page)) == "<class 'pandas.core.frame.DataFrame'>", 'o input deve ser um pandas.dataframe'    

        for row, content in data_instagram_page[data_instagram_page.instagram_followers.isnull() == False].T.iteritems():
            data_instagram_page.loc[row, 'instagram_posts'] = self.processQntOnString(content['instagram_posts'][:content['instagram_posts'].find(' ')].replace(',', ''))
            data_instagram_page.loc[row, 'instagram_followers'] = self.processQntOnString(content['instagram_followers'][:content['instagram_followers'].find(' ')].replace(',', '.'))
            data_instagram_page.loc[row, 'instagram_following'] = self.processQntOnString(content['instagram_following'][:content['instagram_following'].find(' ')].replace(',', '.'))
        return data_instagram_page

    def processQntOnString(self, qnt_string):
        '''
            Nas quantidade podera contar 'k' ou 'm' para mil ou milhoes
        '''
        try:
            key = ''.join(filter(str.isalpha, qnt_string))
            total_chars = '0'*6 if key == 'm' else '0'*3 if key == 'k' else '0'*3 if key == 'mil' else '0'*6 if key == 'mi' else None  

            if total_chars != None and key in qnt_string and '.' in qnt_string:
                #verifica quantos numeros apos o ponto flutuante e retira essa quantidade de zeros
                total_chars = total_chars[len(qnt_string[qnt_string.find('.')+1:qnt_string.find('m')]):]
                qnt_string = qnt_string.replace(key, total_chars).replace('.','')
                return int(qnt_string)
            elif total_chars != None and key in qnt_string:
                qnt_string = qnt_string.replace(key, total_chars)
                return int(qnt_string)
            else:
                return int(qnt_string.replace('.', ''))
        except:
            return 'Error in value: {}'.format(qnt_string)
        
    def cleanFacebookData(self, data_business):
        data_facacebook_page = data_business
        for row, content in data_facacebook_page[data_facacebook_page.facebook_page_name.isnull() == False].T.iteritems():
            if str(content['facebook_likes']) != 'nan': data_facacebook_page.loc[row, 'facebook_likes'] = int(content['facebook_likes'][:content['facebook_likes'].find(' ')].replace(',', '').replace('.', ''))
            if str(content['facebook_followers']) != 'nan': data_facacebook_page.loc[row, 'facebook_followers'] = int(content['facebook_followers'][:content['facebook_followers'].find(' ')].replace(',', '').replace('.', ''))
            if str(content['check-ins']) != 'nan': data_facacebook_page.loc[row, 'check-ins'] = int(content['check-ins'][:content['check-ins'].find(' ')].replace(',', '').replace('.', ''))
        
        return data_facacebook_page
    
    def cleanAdsData(self, data_business):
        ads_data = data_business
        
        assert str(type(ads_data)) == "<class 'pandas.core.frame.DataFrame'>", 'o input deve ser um pandas.dataframe'    
        
        months = {'Jan': '01', 'Fev': '02', 'Mar': '03', 'Abr': '04', 'Maio': '05', 'Mai': '05', 'Jun': '06', 
                 'Jul': '07', 'Ago': '08', 'Set': '09', 'Out': '10', 'Nov': '11', 'Dez': '12'}
        
        for row, content in ads_data.loc[(ads_data.active_ads.isnull() == False)][['active_ads', 'created_in']].T.iteritems():
            try:
                ads_data.loc[row, 'active_ads'] = int(''.join(filter(str.isdigit, content['active_ads'])))
            except TypeError: 
                pass
            try:
                data_ = content['created_in'].replace(' de ', '/')
                month_str = data_.split('/')[1]
                print(month_str)
                ads_data.loc[row, 'created_in'] = pd.to_datetime(data_.replace(month_str, months[month_str]))

            except Exception as error:
                print('ERROR IN ROW ({}) [socialMediaScrapper()] function: cleanAdsData()\n'.format(row))
                print(type(error), str(error))
                
        return data_business

In [45]:
# sp = socialMediaScrapper()
# saude_curitiba = sp.googleSearch()
# saude_curitiba = sp.findSocialMediaAccounts(saude_curitiba)
saude_curitiba = sp.getInstagramData(saude_curitiba)
# saude_curitiba = sp.getFacebookData(saude_curitiba)
# saude_curitiba = sp.getAdsData(saude_curitiba)

c:\users\yan\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:211: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

c:\users\yan\appdata\local\programs\python\python37-32\lib\site-packages\bs4\__init__.py:177: UserWarning:

You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.



getting Instagram Data, 0 processed, last username saved: clinica.curitiba
getting Instagram Data, 1 processed, last username saved: chcufpr
getting Instagram Data, 2 processed, last username saved: chcufpr
getting Instagram Data, 5 processed, last username saved: clinicameitan
getting Instagram Data, 6 processed, last username saved: paranaclinicas
getting Instagram Data, 11 processed, last username saved: otorrinoclinica
getting Instagram Data, 17 processed, last username saved: policlinica_sitio_cercado_ofc
getting Instagram Data, 28 processed, last username saved: cendicor_cendicorbaby
getting Instagram Data, 30 processed, last username saved: ortopediasete
getting Instagram Data, 33 processed, last username saved: neoderme
getting Instagram Data, 34 processed, last username saved: clinicalosangelescuritiba
getting Instagram Data, 38 processed, last username saved: oftalmocuritiba
getting Instagram Data, 40 processed, last username saved: clinica_cardiocare
getting Instagram Data, 

getting Instagram Data, 369 processed, last username saved: laboratorio.fa
getting Instagram Data, 372 processed, last username saved: laboratorio.fa
getting Instagram Data, 375 processed, last username saved: fleury
getting Instagram Data, 382 processed, last username saved: hnsgcuritiba
getting Instagram Data, 386 processed, last username saved: unimedcuritibaoficial
getting Instagram Data, 389 processed, last username saved: magrassoficial
getting Instagram Data, 391 processed, last username saved: odontocompany
getting Instagram Data, 393 processed, last username saved: hnsgcuritiba
getting Instagram Data, 394 processed, last username saved: magrassoficial
getting Instagram Data, 396 processed, last username saved: cvrveterinaria
getting Instagram Data, 397 processed, last username saved: magrassoficial


In [46]:
saude_curitiba.head()

,address,classification,details,fone,link,name,position,has_instagram,has_facebook,facebook_link,instagram_link,link_problem,instagram_posts,instagram_followers,instagram_following,instagram_page_name,instagram_bio_text,instagram_bio_link,instagram_username,instagram_link_problem
0,"Alameda Júlia da Costa, 1447","4,7",Centro médico,(41) 3240-2900 ·,http://curitiba.clinicaadventista.org.br/,Clínica Adventista de Curitiba,1,True,True,https://www.facebook.com/clinica.curitiba,https://www.instagram.com/clinica.curitiba/,NaN,143,589,828,Clínica Adventista de Curitiba,"👨🏻‍⚕️ Consultas, exames e procedimentos⭐ Mais ...",https://l.instagram.com/?u=http%3A%2F%2Fcuriti...,clinica.curitiba,NaN
1,"R. Pasteur, 26","3,9",Clínica oftalmológica,(41) 3523-0082 ·,http://www.hc.ufpr.br/,UFPR - Centro de Visão do Hospital de Clínicas,2,True,True,https://www.facebook.com/chcufpr,https://www.instagram.com/chcufpr,NaN,265,5489,8,Complexo HC UFPR,Perfil Oficial do Complexo Hospital de Clínica...,https://l.instagram.com/?u=http%3A%2F%2Ffacebo...,chcufpr,NaN
2,"R. Gen. Carneiro, 181","4,7",Clínica da saúde da mulher,(41) 3360-1800 ·,http://www.hc.ufpr.br/,Maternidade Do Hospital De Clínicas,3,True,True,https://www.facebook.com/chcufpr,https://www.instagram.com/chcufpr,NaN,265,5489,8,Complexo HC UFPR,Perfil Oficial do Complexo Hospital de Clínica...,https://l.instagram.com/?u=http%3A%2F%2Ffacebo...,chcufpr,NaN
3,"R. Dr. Raul Carneiro Filho, 328","4,9",Policlínica,(41) 3333-3322 ·,http://www.clinicamedicamaisvida.com.br/,Clínica Médica Mais Vida,4,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Av. Luiz Xavier, 68","3,2",Clínica oftalmológica,(41) 3322-9537 ·,http://www.clinicatijucas.com.br/,CLÍNICA TIJUCAS Clínica de Oftalmologia e Gine...,5,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# saude_curitiba.to_csv('./Saved Data/Curitiba/saude.csv', index=None)

with_ads_page = data[data.active_ads.isnull() == False]
with_ads_page = with_ads_page.drop(['classification', 'has_instagram','has_facebook', 'link_problem','instagram_link_problem', 'instagram_bio_text','instagram_bio_link', 'facebook_link_problem', 'about_content', 'library_ads_url', 'ads_page_problem'], axis=1)
with_ads_page.sort_values(by=['instagram_posts'], ascending=False)

In [49]:
saude_curitiba.to_excel('./Saved Data/Curitiba/saude.xls', index=None)

# TODO
prints inline:

 * Incluir print output inline para funcao que esta fazendo request para todos os sites em busca das redes sociais
 * para os instagrans e facebook que ja foram processados/encontrados
 

In [ ]:
# saude_curitiba = pd.DataFrame(saude_curitiba)
# saude_curitiba.reset_index(drop=True, inplace=True)
# saude_curitiba.to_csv('./Saved Data/Curitiba/saude.csv', index=None)